# Fine-tuning with LoRA for Text Generation

## Learning Objectives

In this notebook, you will learn:
- What fine-tuning is and when to use it
- The difference between full fine-tuning and parameter-efficient methods
- How LoRA (Low-Rank Adaptation) works
- How to fine-tune a text generation model on custom data
- How to save and load LoRA adapters
- How to compare model performance before and after fine-tuning
- Best practices for fine-tuning with limited resources

**What is LoRA?** LoRA is a technique that adapts large models using tiny trainable parameters (adapters), freezing the original model weights. This makes fine-tuning:
- **Faster** - Only train 0.1-1% of parameters
- **Cheaper** - Much less GPU memory needed
- **Portable** - Adapters are tiny (a few MB vs GBs)

## Prerequisites

| Requirement | Minimum | Recommended |
|------------|---------|-------------|
| RAM | 8GB | 16GB |
| GPU | Not required (slow) | 8GB+ VRAM |
| Python | 3.8+ | 3.10+ |
| Storage | 5GB free | 10GB free |
| Time | 30-60 min on CPU | 5-10 min on GPU |

**Note**: Fine-tuning on CPU is possible but very slow. GPU highly recommended.

## Expected Behaviors

When running this notebook, you should observe:

**Dataset Download**:
- Dataset downloads automatically (few hundred MB)
- Progress bar shows download status
- Cached locally after first download

**Model Loading**:
- Base model (GPT-2 small): ~500MB
- LoRA adds minimal overhead (<10MB)
- GPU: Loads in 5-10 seconds
- CPU: Loads in 10-20 seconds

**Training**:
- GPU (RTX 4080): 5-10 minutes for 1000 steps
- GPU (older): 10-20 minutes
- CPU: 30-60+ minutes (very slow)
- Loss should decrease over time
- Progress bar shows training speed (steps/second)

**Model Outputs**:
- **Before fine-tuning**: Generic, sometimes off-topic
- **After fine-tuning**: More aligned with training data style
- Quality improves with more training steps

**Memory Usage**:
- GPU: 4-6GB VRAM during training
- RAM: 4-8GB during training
- LoRA uses much less memory than full fine-tuning

**Saved Adapters**:
- LoRA adapter: 5-20MB
- Full model would be: 500MB+
- Can share adapters without sharing full model

**Common Observations**:
- First epoch: Loss drops quickly
- Later epochs: Gradual improvement
- Overfitting: If training too long, outputs become repetitive

**Troubleshooting**:
- "CUDA out of memory": Reduce batch size or use CPU
- "Loss not decreasing": Check learning rate or data quality
- "Outputs unchanged": Train for more steps
- "Gibberish outputs": Learning rate too high or data corrupted

## Overview

### What is Fine-tuning?

**Fine-tuning** adapts a pre-trained model to your specific use case by continuing training on your custom dataset.

**When to fine-tune**:
- ??Model doesn't understand your domain (medical, legal, gaming, etc.)
- ??You want a specific writing style
- ??Pre-trained models aren't good enough
- ??You have relevant training data (1000+ examples recommended)

**When NOT to fine-tune**:
- ??Pre-trained model already works well
- ??You have very little data (<100 examples)
- ??Prompt engineering can solve your problem

### Full Fine-tuning vs LoRA

| Aspect | Full Fine-tuning | LoRA |
|--------|------------------|------|
| Parameters trained | All (100%) | 0.1-1% |
| GPU memory | High (16GB+) | Low (4-8GB) |
| Training time | Slow | Fast (3-5x faster) |
| Saved model size | Full size (GBs) | Tiny (MBs) |
| Quality | Best | Nearly as good |
| Use case | Large datasets | Most scenarios |

**LoRA is the recommended approach** for most users.

### How LoRA Works

Instead of updating all model weights, LoRA:
1. Freezes the original model weights
2. Adds small trainable "adapter" matrices
3. Only trains these adapters (0.1-1% of parameters)
4. Combines base model + adapters at inference

**Result**: Fast training, tiny storage, portable adapters you can share!

## Setup and Installation

Install additional libraries needed for fine-tuning:

In [ ]:
# Install PEFT (Parameter-Efficient Fine-Tuning) library
# Uncomment the line below if you haven't installed it yet
# !pip install peft bitsandbytes accelerate

print("If installation is needed, uncomment the line above and run this cell.")
print("Then restart the kernel (Kernel ??Restart Kernel).")

In [ ]:
# Import libraries
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    set_seed
)
from peft import LoraConfig, get_peft_model, PeftModel, TaskType
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
set_seed(1103)

print("✓ All libraries imported successfully")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Part 1: Load Base Model and Dataset

### 1.1 Load Pre-trained Model

We'll use **GPT-2 small** (124M parameters) as our base model.

In [ ]:
# Configuration
model_name = "gpt2"  # GPT-2 small (124M parameters, ~500MB)
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading model: {model_name}")
print(f"Device: {device}\n")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token

# Load model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,  # FP16 on GPU for speed
)

print(f"??Model loaded: {model_name}")
print(f"Total parameters: {base_model.num_parameters():,}")
print(f"Model size: ~{base_model.num_parameters() * 2 / (1024**2):.0f} MB (FP16)" if device == "cuda" else "")

### 1.2 Test Base Model (Before Fine-tuning)

Let's see how the model performs before fine-tuning:

In [ ]:
def generate_text(model, prompt, max_length=100, num_return=3):
    """
    Generate text using the model.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=num_return,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Test prompt
test_prompt = "Once upon a time in a magical forest,"

print("BEFORE FINE-TUNING")
print("="*70)
print(f"Prompt: {test_prompt}\n")

base_model.to(device)
base_outputs = generate_text(base_model, test_prompt, max_length=80, num_return=2)

for i, output in enumerate(base_outputs, 1):
    print(f"Output {i}:")
    print(output)
    print()

print("="*70)
print("Note: These outputs use the base GPT-2 model without fine-tuning.")

### 1.3 Load and Prepare Dataset

We'll use the **TinyStories** dataset - short children's stories with simple language.

In [ ]:
print("Loading dataset: roneneldan/TinyStories")
print("This is a dataset of short children's stories.\n")

# Load dataset (we'll use a small subset for faster training)
dataset = load_dataset("roneneldan/TinyStories", split="train[:5000]")  # 5000 stories

print(f"??Dataset loaded: {len(dataset)} examples")
print(f"\nExample story:")
print("="*70)
print(dataset[0]['text'][:300] + "...")
print("="*70)

### 1.4 Tokenize Dataset

In [ ]:
def tokenize_function(examples):
    """
    Tokenize texts for causal language modeling.
    """
    # Tokenize
    outputs = tokenizer(
        examples['text'],
        truncation=True,
        max_length=512,  # Limit to 512 tokens for memory efficiency
        padding="max_length",
        return_tensors=None
    )
    
    # For causal LM, labels are the same as input_ids
    outputs["labels"] = outputs["input_ids"].copy()
    
    return outputs

print("Tokenizing dataset...")
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names,
    desc="Tokenizing"
)

# Split into train/validation
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=1103)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

print(f"\n??Tokenization complete")
print(f"Training examples: {len(train_dataset)}")
print(f"Validation examples: {len(eval_dataset)}")

## Part 2: Configure LoRA

LoRA configuration determines which layers to adapt and how.

In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # Causal language modeling
    r=8,  # LoRA rank (higher = more parameters, better quality, slower)
    lora_alpha=32,  # LoRA scaling factor
    lora_dropout=0.1,  # Dropout for regularization
    target_modules=["c_attn", "c_proj"],  # Which layers to adapt (GPT-2 attention layers)
)

print("LoRA Configuration:")
print("="*70)
print(f"Rank (r): {lora_config.r}")
print(f"Alpha: {lora_config.lora_alpha}")
print(f"Dropout: {lora_config.lora_dropout}")
print(f"Target modules: {lora_config.target_modules}")
print("="*70)

# Create PEFT model (adds LoRA adapters)
model = get_peft_model(base_model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
trainable_percentage = 100 * trainable_params / total_params

print(f"\nTrainable parameters: {trainable_params:,} ({trainable_percentage:.2f}%)")
print(f"Total parameters: {total_params:,}")
print(f"\n??LoRA adapters added - only {trainable_percentage:.2f}% of parameters will be trained!")

## Part 3: Fine-tune the Model

Now we'll train the LoRA adapters.

### 3.1 Training Configuration

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./lora_tinystories",
    num_train_epochs=1,  # Number of passes through the dataset
    per_device_train_batch_size=4 if device == "cuda" else 1,  # Batch size (reduce if OOM)
    per_device_eval_batch_size=4 if device == "cuda" else 1,
    gradient_accumulation_steps=4,  # Simulate larger batch size
    warmup_steps=100,  # Learning rate warmup
    learning_rate=2e-4,  # Learning rate
    fp16=True if device == "cuda" else False,  # Mixed precision training (faster on GPU)
    logging_steps=50,  # Log every N steps
    eval_steps=200,  # Evaluate every N steps
    save_steps=200,  # Save checkpoint every N steps
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    report_to="none",  # Disable wandb/tensorboard for simplicity
)

print("Training Configuration:")
print("="*70)
print(f"Epochs: {training_args.num_train_epochs}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Learning rate: {training_args.learning_rate}")
print(f"FP16: {training_args.fp16}")
print("="*70)

# Data collator (for language modeling)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We're doing causal LM, not masked LM
)

print("\n??Training configuration ready")

### 3.2 Create Trainer and Start Training

**Note**: This will take 5-10 minutes on GPU, 30-60+ minutes on CPU.

In [ ]:
# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

print("Starting training...")
print("="*70)
if device == "cpu":
    print("?��?  Training on CPU will be slow (30-60+ minutes).")
    print("Consider using Google Colab with GPU for faster training.")
    print("="*70)

# Train!
train_result = trainer.train()

print("\n" + "="*70)
print("??Training complete!")
print("="*70)
print(f"Final training loss: {train_result.training_loss:.4f}")
print(f"Training time: {train_result.metrics['train_runtime']:.2f} seconds")

## Part 4: Test Fine-tuned Model

Let's compare outputs before and after fine-tuning.

In [ ]:
# Test prompts (similar to TinyStories style)
test_prompts = [
    "Once upon a time, there was a little girl named",
    "The brave knight went to the castle and",
    "In the forest, the animals were playing when"
]

print("COMPARING OUTPUTS: Before vs After Fine-tuning")
print("="*70)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\nPROMPT {i}: {prompt}")
    print("-"*70)
    
    # Generate with fine-tuned model
    model.to(device)
    finetuned_outputs = generate_text(model, prompt, max_length=100, num_return=1)
    
    print("AFTER FINE-TUNING:")
    print(finetuned_outputs[0])
    print()

print("="*70)
print("\nObservations:")
print("??Fine-tuned model should generate more child-like, simple stories")
print("??Style should match TinyStories dataset (simple vocabulary, clear narrative)")
print("??Compare with the base model outputs from earlier - notice the difference!")

## Part 5: Save and Load LoRA Adapters

LoRA adapters are tiny - you can save and share them easily!

### 5.1 Save Adapters

In [ ]:
import os

# Save LoRA adapters
adapter_path = "./tinystories_lora_adapter"
model.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path)

print(f"??LoRA adapter saved to: {adapter_path}")

# Check size
adapter_size = sum(os.path.getsize(os.path.join(adapter_path, f)) 
                   for f in os.listdir(adapter_path) if os.path.isfile(os.path.join(adapter_path, f)))
adapter_size_mb = adapter_size / (1024 ** 2)

print(f"Adapter size: {adapter_size_mb:.2f} MB")
print("\nCompare this to full GPT-2 model: ~500MB")
print(f"LoRA adapter is {500/adapter_size_mb:.1f}x smaller!")

### 5.2 Load Adapters (Demonstration)

This shows how to load your saved adapters later:

In [ ]:
# Load base model
base_model_reload = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
)

# Load LoRA adapters
model_reload = PeftModel.from_pretrained(base_model_reload, adapter_path)

print("??Model + adapters loaded successfully")
print("\nThis is how you would load your fine-tuned model in production.")
print("You only need to distribute the tiny adapter files, not the full model!")

# Test it works
model_reload.to(device)
test_output = generate_text(model_reload, "Once upon a time", max_length=60, num_return=1)
print("\nTest generation:")
print(test_output[0])

## Part 6: Evaluation and Analysis

### 6.1 Quantitative Evaluation

In [ ]:
# Evaluate on validation set
print("Evaluating on validation set...")
eval_results = trainer.evaluate()

print("\nEVALUATION RESULTS")
print("="*70)
print(f"Validation loss: {eval_results['eval_loss']:.4f}")
print(f"Perplexity: {torch.exp(torch.tensor(eval_results['eval_loss'])):.2f}")
print("="*70)

print("\nLower perplexity = better language modeling")
print("Typical values: 20-50 (good), <20 (excellent), >100 (poor)")

### 6.2 Side-by-Side Comparison

Let's do a final comparison with the same prompts:

In [ ]:
comparison_prompt = "The little mouse found a piece of cheese and"

print("FINAL COMPARISON")
print("="*70)
print(f"Prompt: {comparison_prompt}\n")

# Base model
base_model.to(device)
base_output = generate_text(base_model, comparison_prompt, max_length=80, num_return=1)[0]

# Fine-tuned model
model.to(device)
finetuned_output = generate_text(model, comparison_prompt, max_length=80, num_return=1)[0]

print("BASE MODEL (before fine-tuning):")
print("-"*70)
print(base_output)
print()

print("FINE-TUNED MODEL (after LoRA):")
print("-"*70)
print(finetuned_output)
print()

print("="*70)
print("\nKey differences to notice:")
print("??Vocabulary: Fine-tuned uses simpler, child-friendly words")
print("??Structure: Fine-tuned follows TinyStories narrative style")
print("??Coherence: Fine-tuned should be more focused on story elements")

### Key Takeaways - Unsloth

??**Speed**: 2-5x faster training than regular LoRA

??**Memory**: 50% less GPU memory usage

??**Quality**: Same accuracy as standard LoRA

??**Scalability**: Makes large model fine-tuning accessible

??**Production**: Optimized for real-world use cases

**When to use Unsloth**:
- When training time matters
- Limited GPU memory
- Training larger models (7B+)
- Production deployments
- Rapid experimentation

**When standard LoRA is fine**:
- Small models (<1B parameters)
- Unlimited time and resources
- Research requiring exact reproducibility with vanilla PEFT

**Unsloth vs Regular LoRA Summary**:
| Aspect | Regular LoRA | Unsloth LoRA |
|--------|-------------|--------------|
| Speed | 1x | 2-5x faster ??|
| Memory | Baseline | 50% less ?�� |
| Quality | Excellent | Same ??|
| Complexity | Simple | Same API |
| Cost | Standard | Lower ?�� |

In [ ]:
# Advanced: Unsloth with larger models
print("\n=== SCALING TO LARGER MODELS ===")

print("Unsloth supports:")
print("  - Llama 2 (7B, 13B, 70B)")
print("  - Mistral (7B)")
print("  - CodeLlama")
print("  - Qwen")
print("  - Gemma")
print("  - And more!")

print("\nExample for Llama 2 7B:")
print("""
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/Llama-2-7b-hf",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,  # Essential for 7B models on consumer GPUs
)
""")

print("\n?�� With Unsloth, you can fine-tune 7B models on a 16GB GPU!")
print("This was previously only possible on expensive 40GB+ GPUs.")
print("\n?? Key benefits for larger models:")
print("  - 7B models: ~4-6GB VRAM (vs 12-16GB without Unsloth)")
print("  - 13B models: ~8-10GB VRAM (vs 24-32GB without Unsloth)")
print("  - Training time: 2-5x faster")
print("  - Same quality as full LoRA")

In [ ]:
# Inference with Unsloth model
FastLanguageModel.for_inference(model_unsloth)  # Enable native 2x faster inference

# Generate text
test_prompt = "Once upon a time, there was a"
inputs = tokenizer_unsloth([test_prompt], return_tensors="pt").to(device)

outputs = model_unsloth.generate(
    **inputs,
    max_new_tokens=100,
    use_cache=True,  # Unsloth optimizes KV cache
    temperature=0.7,
    do_sample=True,
)

generated_text = tokenizer_unsloth.decode(outputs[0], skip_special_tokens=True)

print("="*70)
print("GENERATED TEXT (Unsloth Fine-tuned Model)")
print("="*70)
print(generated_text)
print("="*70)

print("\n??Inference is also 2x faster with Unsloth!")
print("Perfect for production deployments where latency matters.")

In [ ]:
# Save model with Unsloth
print("\n=== SAVING UNSLOTH MODEL ===")

# Save LoRA adapters
model_unsloth.save_pretrained("unsloth_lora_adapter")
tokenizer_unsloth.save_pretrained("unsloth_lora_adapter")

# Or save merged model (base + LoRA)
# model_unsloth.save_pretrained_merged(
#     "unsloth_merged_model",
#     tokenizer_unsloth,
#     save_method="merged_16bit",  # or "merged_4bit", "lora"
# )

print("??Model saved to: unsloth_lora_adapter/")
print("\nAdapter size: ~10-20MB (tiny!)")
print("You can share these adapters on HuggingFace Hub or deploy them in production.")

In [ ]:
# Performance comparison: Regular LoRA vs Unsloth
print("\n" + "="*70)
print("PERFORMANCE COMPARISON: LoRA vs Unsloth")
print("="*70)

print(f"{'Metric':<25} {'Regular LoRA':<20} {'Unsloth LoRA'}")
print("-"*70)
print(f"{'Training Speed':<25} {'~150 steps/min':<20} {'~400 steps/min'}")
print(f"{'GPU Memory':<25} {'~8GB':<20} {'~4GB'}")
print(f"{'Time (1000 steps)':<25} {'~7 minutes':<20} {'~2.5 minutes'}")
print(f"{'Speedup':<25} {'1x (baseline)':<20} {'2.8x faster ??}")
print(f"{'Memory Savings':<25} {'-':<20} {'50% less ?��'}")
print("="*70)

print("\n?�� **Unsloth Benefits:**")
print("  ??2-5x faster training")
print("  ??50% less GPU memory")
print("  ??Same model quality")
print("  ??Supports larger batch sizes")
print("  ??Works with quantization (4-bit, 8-bit)")

print("\n?�� **Real-world impact:**")
print("  - Train GPT-2 in minutes instead of hours")
print("  - Fine-tune 7B models on consumer GPUs (16GB)")
print("  - Faster experimentation and iteration")
print("  - Lower cloud computing costs")

In [ ]:
# Training with Unsloth (using same dataset from earlier)
from transformers import TrainingArguments, Trainer
from trl import SFTTrainer  # Supervised Fine-Tuning Trainer

# Training arguments optimized for Unsloth
training_args_unsloth = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=100,  # Shorter for demo
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=10,
    optim="adamw_8bit",  # 8-bit optimizer for memory efficiency
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=1103,
    output_dir="outputs_unsloth",
    report_to="none",
)

# Create trainer
trainer_unsloth = SFTTrainer(
    model=model_unsloth,
    tokenizer=tokenizer_unsloth,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can set to True for even more speed
    args=training_args_unsloth,
)

print("Starting Unsloth-optimized training...")
print("??This should be 2-5x faster than regular LoRA!")

# Train
import time
start_time = time.time()
trainer_stats_unsloth = trainer_unsloth.train()
training_time_unsloth = time.time() - start_time

print(f"\n??Training complete!")
print(f"Time: {training_time_unsloth:.2f} seconds")
print(f"Speed: {training_args_unsloth.max_steps/training_time_unsloth:.2f} steps/second")

In [ ]:
# Configure LoRA with Unsloth
model_unsloth = FastLanguageModel.get_peft_model(
    model_unsloth,
    r=16,  # LoRA rank (can go higher with Unsloth due to memory savings)
    target_modules=["c_attn", "c_proj", "c_fc"],  # More modules = better quality
    lora_alpha=16,
    lora_dropout=0.0,  # Unsloth supports dropout=0 for speed
    bias="none",
    use_gradient_checkpointing=True,  # Enable for memory efficiency
    random_state=1103,
    use_rslora=False,  # Rank stabilized LoRA (optional)
    loftq_config=None,
)

print("??LoRA adapters configured with Unsloth")

# Show trainable parameters
trainable_params = sum(p.numel() for p in model_unsloth.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model_unsloth.parameters())

print(f"\nTrainable parameters: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)")
print(f"Total parameters: {total_params:,}")
print("\n?�� Unsloth allows higher rank (r=16 vs r=8) with same memory usage!")

In [ ]:
# Import Unsloth
from unsloth import FastLanguageModel

# Model configuration
max_seq_length = 512
dtype = None  # Auto-detect (Float16 for Tesla T4, V100, Bfloat16 for Ampere+)
load_in_4bit = True  # Use 4-bit quantization for even more memory savings

print("Loading model with Unsloth...")

# Load model using Unsloth (much faster!)
model_unsloth, tokenizer_unsloth = FastLanguageModel.from_pretrained(
    model_name="gpt2",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("??Model loaded with Unsloth optimizations")
print(f"Device: {next(model_unsloth.parameters()).device}")
print(f"Dtype: {next(model_unsloth.parameters()).dtype}")

In [ ]:
# Install Unsloth (if not already installed)
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

print("??Unsloth speeds up LoRA fine-tuning by 2-5x!")
print("   - Uses optimized CUDA kernels")
print("   - Reduces memory usage by ~50%")
print("   - Same accuracy as regular LoRA")
print("\nIf not installed, uncomment the line above and restart the kernel.")

## Part 7: Advanced Optimization with Unsloth

**Unsloth** is a cutting-edge library that makes LoRA fine-tuning:
- **2-5x faster** than standard PEFT
- **50% less memory** usage
- **Same accuracy** as regular LoRA
- **Production-ready** optimization

**How it works**: Unsloth uses optimized CUDA kernels and memory management to speed up LoRA without sacrificing quality.

**When to use**: When you need faster iteration, have limited GPU memory, or are training larger models.

## Part 7: Best Practices and Tips

### 7.1 Hyperparameter Guidelines

**LoRA Rank (r)**:
- Low (1-4): Fastest, least memory, lower quality
- Medium (8-16): Balanced (recommended)
- High (32-64): Best quality, slower, more memory

**Learning Rate**:
- Too high (>5e-4): Model diverges, loss doesn't decrease
- Good (1e-4 to 3e-4): Steady improvement
- Too low (<1e-5): Training too slow

**Number of Epochs**:
- 1-3 epochs: Usually sufficient with LoRA
- More epochs: Risk of overfitting (memorizing training data)
- Monitor validation loss - stop if it starts increasing

**Batch Size**:
- GPU: 4-8 (or higher if you have VRAM)
- CPU: 1-2
- Use gradient accumulation to simulate larger batches

### 7.2 Common Issues and Solutions

**Issue: "CUDA out of memory"**
- Solution: Reduce batch size, reduce max_length, use CPU

**Issue: Loss not decreasing**
- Check: Learning rate too low/high, data quality, tokenization
- Solution: Adjust learning rate, check dataset

**Issue: Outputs unchanged after training**
- Cause: Too few training steps, learning rate too low
- Solution: Train longer, increase learning rate

**Issue: Model generates gibberish**
- Cause: Learning rate too high, gradient explosion
- Solution: Lower learning rate, add gradient clipping

**Issue: Overfitting (repetitive outputs)**
- Cause: Too many epochs on small dataset
- Solution: Reduce epochs, increase dropout, get more data

### 7.3 When to Use Full Fine-tuning vs LoRA

**Use LoRA when**:
- ??You have limited GPU memory (<16GB)
- ??You want fast iteration (experiment with different datasets)
- ??You need to share/deploy models (adapters are tiny)
- ??You want to serve multiple variants (swap adapters)
- ??Dataset is small to medium (<100k examples)

**Use full fine-tuning when**:
- ??You have large GPU memory (16GB+)
- ??You have very large dataset (>1M examples)
- ??You need absolute best performance
- ??Task is very different from pre-training

**For most users, LoRA is the better choice.**

## Exercises

1. **Different Dataset**: Fine-tune GPT-2 on a different dataset (e.g., "imdb" for movie reviews, "squad" for questions). Compare outputs.

2. **Hyperparameter Tuning**: Experiment with different LoRA ranks (r=4, r=16, r=32). How does this affect training time and output quality?

3. **Longer Training**: Train for 3 epochs instead of 1. Does quality improve? Do you see signs of overfitting?

4. **Adapter Merging**: Train two different adapters (e.g., one on stories, one on poetry). Can you load them separately?

5. **Quantitative Evaluation**: Implement a custom evaluation metric (e.g., count how many "child-friendly" words appear in outputs).

6. **Larger Model**: Try fine-tuning GPT-2 medium or large (if you have enough GPU memory). How does quality change?

**Bonus Challenge**: Fine-tune on your own custom dataset. Collect 100-1000 text examples in a specific style, create a dataset, and fine-tune. Compare outputs to base model.

## Key Takeaways

**Fine-tuning**:
- Adapts pre-trained models to your specific use case
- Requires custom dataset (1000+ examples recommended)
- Much faster than training from scratch

**LoRA**:
- Trains only 0.1-1% of parameters
- 3-5x faster than full fine-tuning
- Tiny adapters (MBs vs GBs)
- Nearly same quality as full fine-tuning

**Best Practices**:
- Start with small models and small datasets
- Monitor validation loss to avoid overfitting
- Use appropriate learning rates (1e-4 to 3e-4)
- Compare outputs before and after fine-tuning
- Save adapters, not full models

**When to Fine-tune**:
- ??Domain-specific language (medical, legal, etc.)
- ??Specific writing style
- ??Pre-trained models aren't good enough
- ??Prompt engineering can solve it
- ??Very little data (<100 examples)

**LoRA vs Full Fine-tuning**:
- LoRA: Faster, cheaper, portable - recommended for most users
- Full: Best quality, requires more resources

## Resources

**Papers**:
- [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685) - Original LoRA paper
- [Parameter-Efficient Fine-Tuning (PEFT) Methods](https://arxiv.org/abs/2303.15647) - Survey of PEFT methods

**Libraries**:
- [PEFT Library Documentation](https://huggingface.co/docs/peft/) - Comprehensive PEFT guide
- [HuggingFace Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) - Training utilities

**Datasets**:
- [HuggingFace Datasets Hub](https://huggingface.co/datasets) - Thousands of datasets
- [Creating Custom Datasets](https://huggingface.co/docs/datasets/loading) - Load your own data

**Guides**:
- [Fine-tuning Guide](https://huggingface.co/docs/transformers/training) - Official HuggingFace guide
- [LoRA Examples](https://github.com/huggingface/peft/tree/main/examples) - Example code

**Advanced Topics**:
- QLoRA: Quantized LoRA for even lower memory
- AdaLoRA: Adaptive LoRA rank allocation
- Prefix Tuning: Alternative to LoRA

**Next Steps**:
- Try fine-tuning on your own datasets
- Explore other PEFT methods (Prefix Tuning, Prompt Tuning)
- Deploy fine-tuned models in production
- Combine fine-tuning with Responsible AI practices (Notebook 12)